this chapter is best to use [databricks community cluster](https://community.cloud.databricks.com) to practise

see `../SparkSQL.sql`  and `../SparkSQL.html`

In [1]:
from pyspark.sql import SparkSession, SQLContext
import pyspark.sql.functions as F
from pyspark.sql.types import *

spark = SparkSession\
    .builder\
    .appName("chapter-10-data-src")\
    .getOrCreate()

In [2]:
SPARK_BOOK_DATA_PATH = "/home/gong/spark/books/Spark-The-Definitive-Guide"

In [3]:
file_path = SPARK_BOOK_DATA_PATH + "/data/flight-data/json/2015-summary.json"

In [4]:
spark.read.json(file_path)\
  .createOrReplaceTempView("some_sql_view") # DF => SQL

In [5]:
df = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count)
FROM some_sql_view GROUP BY DEST_COUNTRY_NAME
""")\
  .where("DEST_COUNTRY_NAME like 'S%'").where("`sum(count)` > 10")
# SQL => DF


# COMMAND ----------

In [6]:
df.show(5)

+--------------------+----------+
|   DEST_COUNTRY_NAME|sum(count)|
+--------------------+----------+
|             Senegal|        40|
|              Sweden|       118|
|               Spain|       420|
|    Saint Barthelemy|        39|
|Saint Kitts and N...|       139|
+--------------------+----------+
only showing top 5 rows

